In [1]:
#Imports
import pandas as pd
import numpy as np
from datetime import datetime
from sqlalchemy import text
import matplotlib.pyplot as plt
import seaborn as sns
from Database.database import engine, SessionLocal
from Database.models import FactUserAnalyticsSnapshot

# Configuration of display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Imported successfully")

Imported successfully


In [2]:
#Loading RFM + Survival data for CLV calculation
snapshot_date_key = int(datetime.now().strftime("%Y%m%d"))

print("="*80)
print("LOADING DATA FOR CLV ESTIMATION")
print("="*80)

print("Loading data from fact_user_analytics_snapshot...")
with SessionLocal() as session:
    records = session.query(FactUserAnalyticsSnapshot).filter(
        FactUserAnalyticsSnapshot.snapshot_date_key == snapshot_date_key,
        FactUserAnalyticsSnapshot.subscription_plan_key.in_([2, 3, 4, 5])  # Premium only
    ).all()
    
    data = [{
        'user_key': r.user_key,
        'subscription_plan_key': r.subscription_plan_key,
        'rfm_recency': r.rfm_recency,
        'rfm_frequency': r.rfm_frequency,
        'rfm_monetary': r.rfm_monetary,
        'segment_label': r.segment_label,
        'engagement_level': r.engagement_level,
        'churn_probability': r.churn_probability if r.churn_probability is not None else 0.5,
        'churn_risk_band': r.churn_risk_band,
        'survival_median_time_to_downgrade': r.survival_median_time_to_downgrade if r.survival_median_time_to_downgrade is not None else 180,
        'survival_risk_90d': r.survival_risk_90d if r.survival_risk_90d is not None else 0.5
    } for r in records]

df = pd.DataFrame(data)

print(f"Loaded {len(df):,} premium users")

print("\nData Summary:")
print(f"  RFM Monetary range:              ${df['rfm_monetary'].min():.2f} - ${df['rfm_monetary'].max():.2f}")
print(f"  Avg RFM Monetary:                ${df['rfm_monetary'].mean():.2f}")
print(f"  Avg Survival Time:               {df['survival_median_time_to_downgrade'].mean():.1f} days")
print(f"  Avg Churn Probability:           {df['churn_probability'].mean():.3f}")

LOADING DATA FOR CLV ESTIMATION
Loading data from fact_user_analytics_snapshot...
Loaded 802 premium users

Data Summary:
  RFM Monetary range:              $509.95 - $629.90
  Avg RFM Monetary:                $591.39
  Avg Survival Time:               854.7 days
  Avg Churn Probability:           0.438


In [3]:
#Calculating average revenue per user (ARPU)
print("="*80)
print("CALCULATING AVERAGE REVENUE PER USER (ARPU)")
print("="*80)

# Mapping subscription plans to monthly prices
subscription_prices = {
    1: 0.00,      # Free
    2: 14.99,     # Standard Monthly
    3: 12.50,     # Standard Annual (149.99/12)
    4: 29.99,     # Premium Monthly
    5: 25.00      # Premium Annual (299.99/12)
}

df['monthly_price'] = df['subscription_plan_key'].map(subscription_prices)

print("Subscription Plan Pricing (Monthly):")
for plan_key, price in subscription_prices.items():
    count = len(df[df['subscription_plan_key'] == plan_key])
    if count > 0:
        plan_names = {1: 'Free', 2: 'Standard Monthly', 3: 'Standard Annual', 
                      4: 'Premium Monthly', 5: 'Premium Annual'}
        print(f"  {plan_names[plan_key]:20s}: ${price:6.2f}/month ({count:,} users)")

arpu = df['monthly_price'].mean()
print(f"\nOverall ARPU: ${arpu:.2f} per month")

print("\nARPU by Engagement Level:")
arpu_by_segment = df.groupby('engagement_level')['monthly_price'].agg(['mean', 'count'])
arpu_by_segment.columns = ['Avg_Monthly_Revenue', 'User_Count']
arpu_by_segment = arpu_by_segment.sort_values('Avg_Monthly_Revenue', ascending=False)
print(arpu_by_segment.round(2))


CALCULATING AVERAGE REVENUE PER USER (ARPU)
Subscription Plan Pricing (Monthly):
  Standard Monthly    : $ 14.99/month (204 users)
  Standard Annual     : $ 12.50/month (211 users)
  Premium Monthly     : $ 29.99/month (195 users)
  Premium Annual      : $ 25.00/month (192 users)

Overall ARPU: $20.38 per month

ARPU by Engagement Level:
                  Avg_Monthly_Revenue  User_Count
engagement_level                                 
Dormant                         20.53         184
Highly Engaged                  20.45         202
Medium Engaged                  20.38         254
At Risk                         20.12         162


In [4]:
#Calculating expected customer lifetime
print("="*80)
print("CALCULATING EXPECTED CUSTOMER LIFETIME")
print("="*80)

# Method 1: Survival analysis median time
df['expected_lifetime_months_survival'] = df['survival_median_time_to_downgrade'] / 30

# Method 2: Churn probability to estimate lifetime
# Expected lifetime = 1 / churn_rate
# Adjust churn probability to monthly rate
df['monthly_churn_rate'] = df['churn_probability'] / 12  
df['expected_lifetime_months_churn'] = 1 / df['monthly_churn_rate'].replace(0, 0.01)

# Capping unrealistic lifetimes (such as > 10 years)
df['expected_lifetime_months_churn'] = df['expected_lifetime_months_churn'].clip(upper=120)

df['expected_lifetime_months'] = (
    0.7 * df['expected_lifetime_months_survival'] + 
    0.3 * df['expected_lifetime_months_churn']
)

print("Expected Lifetime Statistics (in months):")
print(f"  Mean:   {df['expected_lifetime_months'].mean():.1f} months")
print(f"  Median: {df['expected_lifetime_months'].median():.1f} months")
print(f"  Min:    {df['expected_lifetime_months'].min():.1f} months")
print(f"  Max:    {df['expected_lifetime_months'].max():.1f} months")

print("\nLifetime Distribution:")
lifetime_bins = pd.cut(df['expected_lifetime_months'], bins=[0, 6, 12, 24, 120], 
                       labels=['< 6 months', '6-12 months', '12-24 months', '> 24 months'])
print(lifetime_bins.value_counts().sort_index())

CALCULATING EXPECTED CUSTOMER LIFETIME
Expected Lifetime Statistics (in months):
  Mean:   33.4 months
  Median: 34.3 months
  Min:    10.6 months
  Max:    61.6 months

Lifetime Distribution:
expected_lifetime_months
< 6 months        0
6-12 months       9
12-24 months    275
> 24 months     518
Name: count, dtype: int64


In [5]:
#Calculating customer lifetime value
print("="*80)
print("CALCULATING CUSTOMER LIFETIME VALUE (CLV)")
print("="*80)

# Method 1: Simple CLV
# CLV = Monthly Revenue × Expected Lifetime (months)
df['clv_simple'] = df['monthly_price'] * df['expected_lifetime_months']

print("Simple CLV Formula:")
print("CLV = Monthly Revenue × Expected Lifetime (months)")
print(f"\nAverage Simple CLV: ${df['clv_simple'].mean():.2f}")

# Method 2: Discounted CLV 
# Discount rate (annual): 10% → monthly: 0.83%
annual_discount_rate = 0.10
monthly_discount_rate = annual_discount_rate / 12

print(f"\nDiscount Rate: {annual_discount_rate*100:.0f}% annual ({monthly_discount_rate*100:.2f}% monthly)")

# CLV = Monthly_Revenue × [(1 - (1 + r)^(-n)) / r]
# where r = discount rate, n = expected lifetime months
df['clv_discounted'] = df['monthly_price'] * (
    (1 - (1 + monthly_discount_rate) ** (-df['expected_lifetime_months'])) / 
    monthly_discount_rate
)

print("\nDiscounted CLV Formula:")
print("   CLV = Monthly_Revenue × [(1 - (1 + r)^(-n)) / r]")
print("   (Accounts for time value of money)")
print(f"\n   Average Discounted CLV: ${df['clv_discounted'].mean():.2f}")

df['clv_value'] = df['clv_discounted']

print("\n" + "="*80)
print("FINAL CLV STATISTICS")
print("="*80)

print(f"\nCLV Summary:")
print(f"  Mean:   ${df['clv_value'].mean():.2f}")
print(f"  Median: ${df['clv_value'].median():.2f}")
print(f"  Min:    ${df['clv_value'].min():.2f}")
print(f"  Max:    ${df['clv_value'].max():.2f}")
print(f"  Std:    ${df['clv_value'].std():.2f}")

total_clv = df['clv_value'].sum()
print(f"\nTotal Portfolio CLV: ${total_clv:,.2f}")
print(f"   (Expected lifetime revenue from all {len(df):,} premium users)")


CALCULATING CUSTOMER LIFETIME VALUE (CLV)
Simple CLV Formula:
CLV = Monthly Revenue × Expected Lifetime (months)

Average Simple CLV: $689.57

Discount Rate: 10% annual (0.83% monthly)

Discounted CLV Formula:
   CLV = Monthly_Revenue × [(1 - (1 + r)^(-n)) / r]
   (Accounts for time value of money)

   Average Discounted CLV: $586.38

FINAL CLV STATISTICS

CLV Summary:
  Mean:   $586.38
  Median: $509.86
  Min:    $127.04
  Max:    $1439.86
  Std:    $314.05

Total Portfolio CLV: $470,275.94
   (Expected lifetime revenue from all 802 premium users)


In [6]:
#Classifying users into CLV bands
print("="*80)
print("CLASSIFYING USERS INTO CLV BANDS")
print("="*80)

clv_25 = df['clv_value'].quantile(0.25)
clv_75 = df['clv_value'].quantile(0.75)

print(f"CLV Percentiles:")
print(f"  25th percentile: ${clv_25:.2f}")
print(f"  50th percentile: ${df['clv_value'].median():.2f}")
print(f"  75th percentile: ${clv_75:.2f}")

def classify_clv_band(clv):
    if clv >= clv_75:
        return 'High Value'
    elif clv >= clv_25:
        return 'Medium Value'
    else:
        return 'Low Value'

df['clv_band'] = df['clv_value'].apply(classify_clv_band)

print("\nCLV BAND DISTRIBUTION:")
clv_band_dist = df['clv_band'].value_counts()
for band, count in clv_band_dist.items():
    pct = count / len(df) * 100
    avg_clv = df[df['clv_band'] == band]['clv_value'].mean()
    total_value = df[df['clv_band'] == band]['clv_value'].sum()
    print(f"  {band:15s}: {count:,} users ({pct:.1f}%) | Avg CLV: ${avg_clv:,.2f} | Total: ${total_value:,.2f}")

high_value_users = df[df['clv_band'] == 'High Value']
print(f"\nHIGH VALUE SEGMENT DETAILS:")
print(f"  Count: {len(high_value_users):,} users ({len(high_value_users)/len(df)*100:.1f}%)")
print(f"  Avg CLV: ${high_value_users['clv_value'].mean():.2f}")
print(f"  Total Value: ${high_value_users['clv_value'].sum():,.2f}")
print(f"  Avg Lifetime: {high_value_users['expected_lifetime_months'].mean():.1f} months")
print(f"  Avg Churn Risk: {high_value_users['churn_probability'].mean():.2%}")


CLASSIFYING USERS INTO CLV BANDS
CLV Percentiles:
  25th percentile: $346.21
  50th percentile: $509.86
  75th percentile: $796.53

CLV BAND DISTRIBUTION:
  Medium Value   : 400 users (49.9%) | Avg CLV: $526.04 | Total: $210,414.58
  High Value     : 201 users (25.1%) | Avg CLV: $1,053.40 | Total: $211,732.91
  Low Value      : 201 users (25.1%) | Avg CLV: $239.45 | Total: $48,128.45

HIGH VALUE SEGMENT DETAILS:
  Count: 201 users (25.1%)
  Avg CLV: $1053.40
  Total Value: $211,732.91
  Avg Lifetime: 46.3 months
  Avg Churn Risk: 18.67%


In [7]:
#CLV analysis by RFM segment and engagement level
print("="*80)
print("CLV ANALYSIS BY SEGMENT")
print("="*80)

print("\nCLV by RFM Segment Label:")
clv_by_rfm = df.groupby('segment_label').agg({
    'user_key': 'count',
    'clv_value': ['mean', 'sum'],
    'expected_lifetime_months': 'mean',
    'churn_probability': 'mean'
}).round(2)

clv_by_rfm.columns = ['User_Count', 'Avg_CLV', 'Total_CLV', 'Avg_Lifetime_Months', 'Avg_Churn_Prob']
clv_by_rfm = clv_by_rfm.sort_values('Total_CLV', ascending=False)
print(clv_by_rfm)

print("\nCLV by Engagement Level:")
clv_by_engagement = df.groupby('engagement_level').agg({
    'user_key': 'count',
    'clv_value': ['mean', 'sum'],
    'expected_lifetime_months': 'mean'
}).round(2)

clv_by_engagement.columns = ['User_Count', 'Avg_CLV', 'Total_CLV', 'Avg_Lifetime_Months']
clv_by_engagement = clv_by_engagement.sort_values('Total_CLV', ascending=False)
print(clv_by_engagement)

print("\nCLV by Subscription Plan:")
plan_names = {2: 'Standard Monthly', 3: 'Standard Annual', 
              4: 'Premium Monthly', 5: 'Premium Annual'}
df['plan_name'] = df['subscription_plan_key'].map(plan_names)

clv_by_plan = df.groupby('plan_name').agg({
    'user_key': 'count',
    'clv_value': ['mean', 'sum'],
    'expected_lifetime_months': 'mean'
}).round(2)

clv_by_plan.columns = ['User_Count', 'Avg_CLV', 'Total_CLV', 'Avg_Lifetime_Months']
clv_by_plan = clv_by_plan.sort_values('Total_CLV', ascending=False)
print(clv_by_plan)


CLV ANALYSIS BY SEGMENT

CLV by RFM Segment Label:
                            User_Count  Avg_CLV  Total_CLV  Avg_Lifetime_Months  Avg_Churn_Prob
segment_label                                                                                  
Promising Starters                 157   748.64  117537.08                43.12            0.20
Active High-Value Learners          75   891.51   66863.47                51.46            0.15
Recently Churned                   122   472.68   57666.85                26.83            0.51
Engaged Subscribers                 76   701.03   53278.54                43.36            0.21
Loyal Long-Term                     51   879.81   44870.48                52.01            0.14
Declining Engagement                93   427.35   39743.45                22.03            0.76
New Premium Users                   50   554.86   27742.96                30.69            0.35
High-Value at Risk                  69   321.17   22160.72                18.35      

In [8]:
print("="*80)
print("HIGH-VALUE AT-RISK ANALYSIS")
print("="*80)

high_value_at_risk = df[
    (df['clv_band'] == 'High Value') & 
    (df['churn_probability'] > 0.5)
].sort_values('clv_value', ascending=False)

print(f"\nCRITICAL: {len(high_value_at_risk)} high-value users at risk of churning")
print(f"   Potential Revenue Loss: ${high_value_at_risk['clv_value'].sum():,.2f}")

if len(high_value_at_risk) > 0:
    print("\nTop 15 High-Value At-Risk Users:")
    print(high_value_at_risk.head(15)[[
        'user_key',
        'segment_label',
        'clv_value',
        'churn_probability',
        'survival_median_time_to_downgrade',
        'monthly_price'
    ]].to_string(index=False))

avg_clv_at_risk = high_value_at_risk['clv_value'].mean()
retention_cost_per_user = 50  # This is just an assumption - $50 cost to retain a user (campaigns, discounts and so on)
roi_per_user = avg_clv_at_risk - retention_cost_per_user

print(f"\nRETENTION ROI ANALYSIS:")
print(f"  Avg CLV of at-risk users:     ${avg_clv_at_risk:.2f}")
print(f"  Retention cost per user:      ${retention_cost_per_user:.2f}")
print(f"  Net value per saved user:     ${roi_per_user:.2f}")
print(f"  Total potential savings:      ${roi_per_user * len(high_value_at_risk):,.2f}")
print(f"  Recommended retention budget: ${retention_cost_per_user * len(high_value_at_risk):,.2f}")


HIGH-VALUE AT-RISK ANALYSIS

CRITICAL: 4 high-value users at risk of churning
   Potential Revenue Loss: $3,217.63

Top 15 High-Value At-Risk Users:
 user_key    segment_label  clv_value  churn_probability  survival_median_time_to_downgrade  monthly_price
      586 Recently Churned 810.553437           0.573865                               1049          29.99
      117 Recently Churned 806.846037           0.584434                               1047          29.99
      860 Recently Churned 802.410551           0.553404                               1024          29.99
      298 Recently Churned 797.815024           0.568661                               1023          29.99

RETENTION ROI ANALYSIS:
  Avg CLV of at-risk users:     $804.41
  Retention cost per user:      $50.00
  Net value per saved user:     $754.41
  Total potential savings:      $3,017.63
  Recommended retention budget: $200.00


In [9]:
print("="*80)
print("UPDATING DATABASE WITH CLV ESTIMATES")
print("="*80)

update_df = df[['user_key', 'clv_value', 'clv_band']].copy()

print(f"Updating {len(update_df):,} user records...")

updated_count = 0
with SessionLocal() as session:
    for idx, row in update_df.iterrows():
        record = session.query(FactUserAnalyticsSnapshot).filter(
            FactUserAnalyticsSnapshot.user_key == int(row['user_key']),
            FactUserAnalyticsSnapshot.snapshot_date_key == snapshot_date_key
        ).first()
        
        if record:
            record.clv_value = float(row['clv_value'])
            record.clv_band = row['clv_band']
            updated_count += 1
            
            if updated_count % 500 == 0:
                session.commit()
                print(f"  Updated {updated_count:,} records...")
    
    session.commit()

print(f"\nUpdated {updated_count:,} records in fact_user_analytics_snapshot")

with SessionLocal() as session:
    total = session.query(FactUserAnalyticsSnapshot).filter(
        FactUserAnalyticsSnapshot.snapshot_date_key == snapshot_date_key
    ).count()
    
    with_clv_value = session.query(FactUserAnalyticsSnapshot).filter(
        FactUserAnalyticsSnapshot.snapshot_date_key == snapshot_date_key,
        FactUserAnalyticsSnapshot.clv_value.isnot(None)
    ).count()
    
    with_clv_band = session.query(FactUserAnalyticsSnapshot).filter(
        FactUserAnalyticsSnapshot.snapshot_date_key == snapshot_date_key,
        FactUserAnalyticsSnapshot.clv_band.isnot(None)
    ).count()
    
    print("\nVerification:")
    print(f"  Total records:         {total:,}")
    print(f"  With clv_value:        {with_clv_value:,}")
    print(f"  With clv_band:         {with_clv_band:,}")

print("\nDatabase update complete!")


UPDATING DATABASE WITH CLV ESTIMATES
Updating 802 user records...
  Updated 500 records...

Updated 802 records in fact_user_analytics_snapshot

Verification:
  Total records:         1,000
  With clv_value:        802
  With clv_band:         802

Database update complete!
